# 03 — Modeling & Performance (MP)

This notebook is **contract-driven** and assumes upstream steps have been refactored:

- `src/data/preprocess.py` → produces `data/interim/loans_cleaned.parquet`
- `src/features/build_features.py` → produces `data/processed/engineered_features_v1.parquet`
- `notebooks/02_fe.ipynb` → produces `data/artifacts/feature_spec_v1.json`

**Inputs**
- `data/processed/engineered_features_v1.parquet`
- `data/artifacts/feature_spec_v1.json`

**Outputs**
- `models/bundles/<bundle_id>/model.joblib`
- `models/bundles/<bundle_id>/metadata.json`
- `models/bundles/<bundle_id>/feature_spec_v1.json` (copied for provenance)


In [10]:
from __future__ import annotations

import hashlib
import json
from datetime import datetime, timezone
from pathlib import Path
from typing import Any, List

import joblib
import numpy as np
import pandas as pd
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, brier_score_loss, log_loss, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

# Calibration: sklearn >=1.6 uses FrozenEstimator for prefit; fall back gracefully if unavailable.
try:
    from sklearn.frozen import FrozenEstimator
except Exception:  # pragma: no cover
    FrozenEstimator = None  # type: ignore

from sklearn.calibration import CalibratedClassifierCV

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

PROJECT_ROOT = Path.cwd().parent  # notebooks/ -> repo root
DATA_DIR = PROJECT_ROOT / "data"
ARTIFACTS_DIR = DATA_DIR / "artifacts"
PROCESSED_DIR = DATA_DIR / "processed"
BUNDLES_DIR = PROJECT_ROOT / "models" / "bundles"

FEATURE_MATRIX_PATH = PROCESSED_DIR / "engineered_features_v1.parquet"
FEATURE_SPEC_PATH = ARTIFACTS_DIR / "feature_spec_v1.json"

assert FEATURE_MATRIX_PATH.exists(), f"Missing: {FEATURE_MATRIX_PATH}"
assert FEATURE_SPEC_PATH.exists(), f"Missing: {FEATURE_SPEC_PATH}"


def read_json(path: Path) -> Any:
    return json.loads(path.read_text(encoding="utf-8"))


def sha256_bytes(b: bytes) -> str:
    return hashlib.sha256(b).hexdigest()

## 1) Load feature matrix + enforce the feature contract

We use `feature_spec_v1.json` to determine:
- `TARGET` (`spec.target.name`)
- the contracted feature columns, inferred from the spec + observed engineered matrix columns


In [11]:
spec = read_json(FEATURE_SPEC_PATH)
TARGET = spec["target"]["name"]
ALLOWED_TARGET_VALUES = set(spec["target"].get("allowed_values", [0, 1]))

df = pd.read_parquet(FEATURE_MATRIX_PATH)
print("feature_matrix shape:", df.shape)
print("target:", TARGET)

if TARGET not in df.columns:
    raise KeyError(f"Target '{TARGET}' not present in feature matrix.")

y = df[TARGET]
if y.isna().any():
    raise ValueError(
        "Target contains missing values; upstream preprocessing should have excluded unmapped rows."
    )

unique_vals = set(int(v) for v in y.unique())
if not unique_vals.issubset(ALLOWED_TARGET_VALUES):
    raise ValueError(
        f"Target values {sorted(unique_vals)} not subset of allowed {sorted(ALLOWED_TARGET_VALUES)}"
    )


# ----------------------------
# Build contracted feature list from the spec + observed engineered matrix
# ----------------------------

# 1) datetime derived features
datetime_feats: List[str] = []
for _col, cfg in spec.get("features", {}).get("datetime", {}).items():
    datetime_feats.extend(cfg.get("derived_features", []))

# 2) engineered features are named keys in spec.features.engineered
engineered_feats = list(spec.get("features", {}).get("engineered", {}).keys())

# 3) categorical outputs:
categorical_cfg = spec.get("features", {}).get("categorical", {})

cat_one_hot_cols: List[str] = []
cat_scalar_cols: List[str] = []
for col, cfg in categorical_cfg.items():
    strat = cfg.get("encoding_strategy")
    if strat == "one_hot":
        cat_one_hot_cols.extend([c for c in df.columns if c.startswith(f"{col}_")])
    elif strat == "target_mean":
        cat_scalar_cols.append(f"{col}__target_mean")
    else:
        cat_scalar_cols.append(f"{col}__freq")

# 4) numerical outputs: original numeric + optional transformed columns (log1p)
numerical_cfg = spec.get("features", {}).get("numerical", {})
num_cols = list(numerical_cfg.keys())
num_transformed = []
for col, cfg in numerical_cfg.items():
    if cfg.get("planned_transformation") == "log":
        num_transformed.append(f"{col}__log1p")

FEATURE_COLS = sorted(
    set(
        datetime_feats
        + engineered_feats
        + cat_one_hot_cols
        + cat_scalar_cols
        + num_cols
        + num_transformed
    )
)

missing_features = [c for c in FEATURE_COLS if c not in df.columns]
if missing_features:
    raise ValueError(
        f"Feature matrix missing {len(missing_features)} contracted features. Example: {missing_features[:25]}"
    )

X = df[FEATURE_COLS].copy()
y = df[TARGET].astype(int).copy()

print("X shape:", X.shape)
print("Positive rate:", float(y.mean()))

feature_matrix shape: (1370163, 36)
target: default
X shape: (1370163, 31)
Positive rate: 0.21465110355483252


## 2) Construct an anchor timestamp for temporal splitting

Upstream spec drops raw `issue_d`. We reconstruct an anchor using the derived datetime features:
- `issue_d_year` + `issue_d_month` (preferred)
- else `issue_d_year` + `issue_d_quarter`

If neither exists, we fall back to row order (not ideal for time-series evaluation).


In [12]:
def make_anchor_timestamp(df_all: pd.DataFrame) -> pd.Series:
    if "issue_d_year" in df_all.columns and "issue_d_month" in df_all.columns:
        y = df_all["issue_d_year"].astype("Int64")
        m = df_all["issue_d_month"].astype("Int64")
        return pd.to_datetime(pd.DataFrame({"year": y, "month": m, "day": 1}), errors="raise")
    if "issue_d_year" in df_all.columns and "issue_d_quarter" in df_all.columns:
        y = df_all["issue_d_year"].astype("Int64")
        q = df_all["issue_d_quarter"].astype("Int64")
        m = (q - 1) * 3 + 1
        return pd.to_datetime(pd.DataFrame({"year": y, "month": m, "day": 1}), errors="raise")
    return pd.Series(pd.RangeIndex(len(df_all)), index=df_all.index)


df = df.copy()
df["_anchor"] = make_anchor_timestamp(df)
print(df["_anchor"].min(), df["_anchor"].max())

2007-06-01 00:00:00 2018-12-01 00:00:00


## 3) Time-based split (train/val/test)

We sort by `_anchor` to avoid temporal leakage.


In [13]:
def time_based_split(
    df_all: pd.DataFrame,
    X_all: pd.DataFrame,
    y_all: pd.Series,
    anchor_col: str,
    train_frac: float = 0.70,
    val_frac: float = 0.15,
    test_frac: float = 0.15,
):
    if abs(train_frac + val_frac + test_frac - 1.0) > 1e-9:
        raise ValueError("train/val/test fractions must sum to 1.0")

    order = np.argsort(df_all[anchor_col].to_numpy())
    df_s = df_all.iloc[order].reset_index(drop=True)
    X_s = X_all.iloc[order].reset_index(drop=True)
    y_s = y_all.iloc[order].reset_index(drop=True)

    n = len(df_s)
    n_train = int(n * train_frac)
    n_val = int(n * val_frac)

    idx_train = slice(0, n_train)
    idx_val = slice(n_train, n_train + n_val)
    idx_test = slice(n_train + n_val, n)

    def pack(slc):
        return df_s.iloc[slc], X_s.iloc[slc], y_s.iloc[slc]

    df_train, X_train, y_train = pack(idx_train)
    df_val, X_val, y_val = pack(idx_val)
    df_test, X_test, y_test = pack(idx_test)

    def rate(s: pd.Series) -> float:
        return float(pd.Series(s).astype(int).mean()) if len(s) else 0.0

    meta = {
        "train_rows": len(df_train),
        "val_rows": len(df_val),
        "test_rows": len(df_test),
        "train_anchor_min": str(df_train[anchor_col].min()),
        "train_anchor_max": str(df_train[anchor_col].max()),
        "val_anchor_min": str(df_val[anchor_col].min()),
        "val_anchor_max": str(df_val[anchor_col].max()),
        "test_anchor_min": str(df_test[anchor_col].min()),
        "test_anchor_max": str(df_test[anchor_col].max()),
        "train_default_rate": rate(y_train),
        "val_default_rate": rate(y_val),
        "test_default_rate": rate(y_test),
    }
    return df_train, X_train, y_train, df_val, X_val, y_val, df_test, X_test, y_test, meta


df_train, X_train, y_train, df_val, X_val, y_val, df_test, X_test, y_test, split_meta = (
    time_based_split(df, X, y, anchor_col="_anchor")
)
split_meta

{'train_rows': 959114,
 'val_rows': 205524,
 'test_rows': 205525,
 'train_anchor_min': '2007-06-01 00:00:00',
 'train_anchor_max': '2016-04-01 00:00:00',
 'val_anchor_min': '2016-04-01 00:00:00',
 'val_anchor_max': '2017-03-01 00:00:00',
 'test_anchor_min': '2017-03-01 00:00:00',
 'test_anchor_max': '2018-12-01 00:00:00',
 'train_default_rate': 0.19081881820096463,
 'val_default_rate': 0.26563807633171793,
 'test_default_rate': 0.2748814012893809}

## 4) Guardrail: drop all-null columns (train-derived)

Some columns can be all-null in the training window. We derive the drop list from train only and apply to val/test.


In [14]:
def drop_all_null_cols_train_only(Xtr: pd.DataFrame, *others: pd.DataFrame):
    all_null = [c for c in Xtr.columns if Xtr[c].isna().all()]
    Xtr2 = Xtr.drop(columns=all_null)
    outs = [Xtr2]
    for Xo in others:
        outs.append(Xo.drop(columns=[c for c in all_null if c in Xo.columns]))
    return all_null, outs


all_null_cols, (Xtr, Xva, Xte) = drop_all_null_cols_train_only(X_train, X_val, X_test)
print("Dropped all-null cols:", len(all_null_cols))
all_null_cols[:20]

Dropped all-null cols: 0


[]

## 5) Baseline model: Logistic Regression

The engineered feature matrix is numeric already, so the pipeline is:
- median imputation
- scaling
- logistic regression


In [15]:
baseline = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler(with_mean=False)),
        ("clf", LogisticRegression(max_iter=400, class_weight="balanced", random_state=42)),
    ]
)

baseline.fit(Xtr, y_train.to_numpy())
pva_lr = baseline.predict_proba(Xva)[:, 1]

baseline_val_metrics = {
    "roc_auc": float(roc_auc_score(y_val, pva_lr)),
    "pr_auc": float(average_precision_score(y_val, pva_lr)),
    "brier": float(brier_score_loss(y_val, pva_lr)),
    "log_loss": float(log_loss(y_val, pva_lr)),
}
baseline_val_metrics

/Users/mcharris/Developer/mc-harris1/credit-risk-pd/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 400 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=400).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


{'roc_auc': 0.6781507232790471,
 'pr_auc': 0.4181782189708343,
 'brier': 0.24559626173866972,
 'log_loss': 0.6888657027258445}

## 6) Primary model: HistGradientBoostingClassifier

We still impute missing values. HGB is often a strong baseline for tabular risk modeling.


In [16]:
hgb = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        (
            "clf",
            HistGradientBoostingClassifier(
                random_state=42,
                max_depth=6,
                learning_rate=0.05,
                max_iter=300,
                min_samples_leaf=30,
                early_stopping=True,
                scoring="loss",
            ),
        ),
    ]
)

hgb.fit(Xtr, y_train.to_numpy())
pva_hgb = hgb.predict_proba(Xva)[:, 1]

primary_val_metrics = {
    "roc_auc": float(roc_auc_score(y_val, pva_hgb)),
    "pr_auc": float(average_precision_score(y_val, pva_hgb)),
    "brier": float(brier_score_loss(y_val, pva_hgb)),
    "log_loss": float(log_loss(y_val, pva_hgb)),
}
primary_val_metrics

{'roc_auc': 0.6870778716927307,
 'pr_auc': 0.43110134135465245,
 'brier': 0.1786850873834222,
 'log_loss': 0.535503157477358}

## 7) Test evaluation (first touch)

We evaluate once on test.


In [20]:
pte_hgb = hgb.predict_proba(Xte)[:, 1]

test_metrics_raw = {
    "roc_auc": float(roc_auc_score(y_test, pte_hgb)),
    "pr_auc": float(average_precision_score(y_test, pte_hgb)),
    "brier": float(brier_score_loss(y_test, pte_hgb)),
    "log_loss": float(log_loss(y_test, pte_hgb)),
}
test_metrics_raw

{'roc_auc': 0.6957195818441747,
 'pr_auc': 0.44281279473944746,
 'brier': 0.1809689975422927,
 'log_loss': 0.5395366615561296}

## 8) Optional calibration (sigmoid)

We fit a calibrator on the **validation** set only.

If `FrozenEstimator` is unavailable, we skip calibration.


In [ ]:
calibrated = None
calibration_metrics = None

if FrozenEstimator is None:
    print("FrozenEstimator not available in this sklearn version; skipping calibration.")
else:
    cal = CalibratedClassifierCV(FrozenEstimator(hgb), method="sigmoid", cv="prefit")
    cal.fit(Xva, y_val.to_numpy())
    pte_cal = cal.predict_proba(Xte)[:, 1]

    calibration_metrics = {
        "raw_test_brier": float(brier_score_loss(y_test, pte_hgb)),
        "cal_test_brier": float(brier_score_loss(y_test, pte_cal)),
    }
    calibrated = cal
    calibration_metrics

## 9) Freeze + save model bundle

We save:
- `model.joblib`
- `metadata.json`
- `feature_spec_v1.json` (copy)

Bundle ID is time-stamped for reproducibility.


In [18]:
BUNDLES_DIR.mkdir(parents=True, exist_ok=True)

spec_hash = sha256_bytes(FEATURE_SPEC_PATH.read_bytes())
bundle_id = f"pd_hgb_mp__{datetime.now(timezone.utc).strftime('%Y%m%dT%H%M%SZ')}"
bundle_dir = BUNDLES_DIR / bundle_id
bundle_dir.mkdir(parents=True, exist_ok=True)

# select final model
final_model = hgb
final_name = "hgb_uncalibrated"
if calibration_metrics is not None and calibrated is not None:
    if calibration_metrics["cal_test_brier"] < calibration_metrics["raw_test_brier"]:
        final_model = calibrated
        final_name = "hgb_sigmoid_calibrated"

joblib.dump(final_model, bundle_dir / "model.joblib")

metadata = {
    "bundle_id": bundle_id,
    "model_name": final_name,
    "created_at_utc": datetime.now(timezone.utc).isoformat(),
    "inputs": {
        "feature_matrix": str(FEATURE_MATRIX_PATH),
        "feature_spec": str(FEATURE_SPEC_PATH),
    },
    "feature_spec_hash_sha256": spec_hash,
    "split_meta": split_meta,
    "dropped_all_null_cols": all_null_cols,
    "feature_cols": list(Xtr.columns),
    "metrics": {
        "baseline_val": baseline_val_metrics,
        "primary_val": primary_val_metrics,
        "test_raw": test_metrics_raw,
        "calibration": calibration_metrics,
    },
}

(bundle_dir / "metadata.json").write_text(json.dumps(metadata, indent=2), encoding="utf-8")
(bundle_dir / "feature_spec_v1.json").write_text(json.dumps(spec, indent=2), encoding="utf-8")

print("Saved bundle:", bundle_dir)

Saved bundle: /Users/mcharris/Developer/mc-harris1/credit-risk-pd/models/bundles/pd_hgb_mp__20251218T203133Z


Permutation Importance Inspection

In [19]:
from sklearn.inspection import permutation_importance

perm = permutation_importance(
    hgb,
    Xva,
    y_val,
    n_repeats=5,
    random_state=42,
    scoring="roc_auc",
)

importances = pd.DataFrame(
    {
        "feature": Xva.columns,
        "importance_mean": perm.importances_mean,
        "importance_std": perm.importances_std,
    }
).sort_values("importance_mean", ascending=False)
importances.head(20)

,feature,importance_mean,importance_std
26,sub_grade__target_mean,5.783075e-02,1.545753e-04
25,loan_amnt,8.118510e-03,1.926005e-04
2,dti,6.697387e-03,1.840674e-04
3,emp_length__target_mean,6.019127e-03,2.610362e-04
19,home_ownership_RENT,4.563915e-03,1.886532e-04
15,home_ownership_MORTGAGE,4.260504e-03,8.490067e-05
27,term_ 36 months,3.790455e-03,2.076732e-04
0,annual_inc,3.278026e-03,2.219885e-04
12,grade_numeric,3.140799e-03,9.248882e-05
20,int_rate,2.829425e-03,1.472406e-04


Permutation importance inspection revealed no evidence of target leakage or post-outcome signal. Top features align with standard credit risk drivers (pricing, capacity, stability).